In [1]:
import sys
sys.path.append('../src')
from config import *

In [2]:
import melt
melt.init_flags()
FLAGS = melt.get_flags()
FLAGS.pretrained = '../input/tf-xlm-roberta-large'

In [5]:
from model import xlm_model as Model

I0426 13:29:03.503679 140585931941696 file_utils.py:41] PyTorch version 1.4.0 available.
I0426 13:29:03.504853 140585931941696 file_utils.py:57] TensorFlow version 2.1.0 available.


In [6]:
model = Model()

I0426 13:29:15.424333 140585931941696 logging.py:97] load xlm_model from ../input/tf-xlm-roberta-large start
I0426 13:29:15.425956 140585931941696 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-large/config.json
I0426 13:29:15.426978 140585931941696 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_leng

In [7]:
melt.print_model(model)

Model: "tf_roberta_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) ((None, None, 1024), (Non 559890432 
Total params: 559,890,432
Trainable params: 559,890,432
Non-trainable params: 0
_________________________________________________________________
Model: "xlm_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_roberta_model (TFRobertaM ((None, None, 1024), (Non 559890432 
_________________________________________________________________
dense (Dense)                multiple                  1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape             

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, None)]            0         
_________________________________________________________________
xlm_model (XlmModel)         (None, 1)                 559891457 
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.layers[1].summary()

Model: "xlm_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_roberta_model (TFRobertaM ((None, None, 1024), (Non 559890432 
_________________________________________________________________
dense (Dense)                multiple                  1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________


In [17]:
import gezi
from transformers import TFAutoModel
def xlm_model():
  pretrained = FLAGS.pretrained 
  with gezi.Timer(f'load xlm_model from {pretrained}', True, print):
    transformer = TFAutoModel.from_pretrained(pretrained)
  input_word_ids = tf.keras.Input(shape=(FLAGS.max_len,), dtype=tf.int32, name="input_word_ids")
  sequence_output = transformer(input_word_ids)[0]
  cls_token = sequence_output[:, 0, :]
  odim = len(toxic_types) + 1 if FLAGS.multi_head else 1
  out = tf.keras.layers.Dense(odim, activation='sigmoid')(cls_token)

  model = tf.keras.Model(inputs=input_word_ids, outputs=out)

  return model
model = xlm_model()

I0426 13:38:04.556453 140585931941696 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-large/config.json
I0426 13:38:04.557458 140585931941696 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 514,
  "min_length": 0,
  "model_type": "xlm-roberta",
  "no_repeat_ngr

load xlm_model from ../input/tf-xlm-roberta-large start
load xlm_model from ../input/tf-xlm-roberta-large duration: 4.168493270874023


In [21]:
model.layers

In [22]:
root='../working/exps/v2/toxic-multi-en-finetune-unint-large'
model.load_weights(f'{root}/model.h5')

In [30]:
os.system(f'mkdir -p {root}/tf-xlm-roberta-large')

0

In [31]:
model.layers[1].save_pretrained(f'{root}/tf-xlm-roberta-large')

I0426 13:42:54.188905 140585931941696 configuration_utils.py:142] Configuration saved in ../working/exps/v2/toxic-multi-en-finetune-unint-large/tf-xlm-roberta-large/config.json
I0426 13:42:58.403539 140585931941696 modeling_tf_utils.py:246] Model weights saved in ../working/exps/v2/toxic-multi-en-finetune-unint-large/tf-xlm-roberta-large/tf_model.h5


In [35]:
model.layers[3].save_weights('f{root}/dense.h5')

AttributeError: 'Dense' object has no attribute 'save_weights'

In [29]:
model.layers[1].layers[0]

In [9]:
os.system(f'mkdir -p {root}')
model.layers[-1].layers[0].save_pretrained(f'{root}/tf-xlm-roberta-base')

I0426 12:15:21.764256 140167830091584 configuration_utils.py:142] Configuration saved in ../working/exps/xlm/toxic-from-mix/tf-xlm-roberta-base/config.json
I0426 12:15:23.937429 140167830091584 modeling_tf_utils.py:246] Model weights saved in ../working/exps/xlm/toxic-from-mix/tf-xlm-roberta-base/tf_model.h5


In [10]:
model.layers[1].save_weights('../working/exps/v2/toxic-multi-en-finetune-unint/lang_model_weight.h5')

In [11]:
FLAGS.pretrained = '../input/tf-xlm-roberta-large'
model = Model()
model.load_weights('../working/exps/v2/toxic-multi-en-finetune-unint-large/model.h5')

I0426 12:15:28.436675 140167830091584 logging.py:97] load xlm_model from ../input/tf-xlm-roberta-large start
I0426 12:15:28.438077 140167830091584 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-large/config.json
I0426 12:15:28.439062 140167830091584 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_leng

ValueError: You are trying to load a weight file containing 2 layers into a model with 1 layers.

In [ ]:
model.layers[1].save_weights('../working/exps/v2/toxic-multi-en-finetune-unint-large/lang_model_weight.h5')

In [ ]:
model.weights[-1]

In [ ]:
model.load_weights('../working/exps/v1/debug/model_weight.h5')
model.weights[-1]

In [3]:
import gezi
from dataset import Dataset
ds = Dataset('valid').make_batch(32, gezi.list_files('../input/tfrecords/xlm/validation'))
next(iter(ds))

({'identity_hate': <tf.Tensor: shape=(32, 1, 1), dtype=float32, numpy=
  array([[[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]]], dtype=float32)>,
  'input_word_ids': <tf.Tensor: shape=(32, 192), dtype=int32, numpy=
  array([[     0,    152,   8331, ...,      1,      1,      1],
         [     0,   2091,    221, ...,    177,     16,      2],
         [     0, 203145,    126, ...,      1,      1,      1],
         ...,

In [4]:
next(iter(ds))

({'identity_hate': <tf.Tensor: shape=(32, 1, 1), dtype=float32, numpy=
  array([[[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]],
  
         [[0.]]], dtype=float32)>,
  'input_word_ids': <tf.Tensor: shape=(32, 192), dtype=int32, numpy=
  array([[     0,    152,   8331, ...,      1,      1,      1],
         [     0,   2091,    221, ...,    177,     16,      2],
         [     0, 203145,    126, ...,      1,      1,      1],
         ...,

In [ ]:
from evaluate import evaluate
from husky.callbacks import EvalCallback
eval_callback = EvalCallback(model, ds, evaluate)

In [ ]:
eval_callback.eval()

In [ ]:
import tensorflow as tf
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(tf.train.latest_checkpoint('../working/exps/v1/debug'))
model.weights[-1]